In [27]:
import numpy as np
from transitions import Machine

STEP_SIZE = 10
# Note: cooling is positive in GHz and heating is negative in GHz

NUM_OF_LASERS = 8   # number of lasers and/or MRMs
LASER_FREQ_SPACING = 400  # in GHz
LASER_FREQ_UNCERTAINTY = 50  # ± this number in GHz

MRM_FSR = 1800  # in GHz
MRM_FREQ_SPACING = 400  # in GHz
MRM_FREQ_UNCERTAINTY = 100  # in GHz
INITIAL_COOLDOWN = MRM_FSR // 2  # in GHz

# will track the resonances -1*FSR over, 0*FSR over, and 1*FSR over
WHICH_RESONANCES = list(range(-10, 11, 1))
BARREL_ROLL_SEQ = [0, 5, 1, 6, 2, 7, 3, 4]


class Lasers(object):

    def __init__(self,
                 variations=False,
                 corners=False,
                 reverse=False,
                 offset=0) -> None:
        self._delta = None

        # start by having all the laser lines 'bright'
        # will be replaced by the MRM number that captures that laser
        # Note: need to consider upstream vs. downstream
        self._brightness = ['bright'] * NUM_OF_LASERS

        # If reverse, the first laser line will have the highest frequency
        self._reverse = reverse

        # offset specifies the additive frequency offset from 0
        self._offset = offset

        self.populate_laser_lines(variations, corners)        

    def populate_laser_lines(self, variations, corners):
        # the first laser line is always at 0 GHz
        laser_lines = np.arange(
            NUM_OF_LASERS, dtype=int) * LASER_FREQ_SPACING

        if self._reverse:
            laser_lines = np.flip(laser_lines)
        
        laser_lines += self._offset    
    
        if variations:
            if corners:  # only evaluate the corners
                delta = np.random.randint(2, size=laser_lines.size)
                delta = 2*delta - 1  # convert to [-1, +1] random numbers
                delta *= LASER_FREQ_UNCERTAINTY
            else:
                delta = np.random.uniform(-1*LASER_FREQ_UNCERTAINTY,
                                          LASER_FREQ_UNCERTAINTY,
                                          laser_lines.size)
            # round to the nearest STEP_SIZE
            delta = np.round(delta/STEP_SIZE, decimals=0) * STEP_SIZE
            delta = delta.astype(int)

            self._delta = delta
            laser_lines += delta

        self._laser_lines = laser_lines

    def capture_laser_line(self, which_laser, which_ring):
        assert self._brightness[which_laser] == 'bright', \
            f"Laser line {which_laser} (at {self._laser_lines[which_laser]} GHz) should be bright to be captured"
        # which_ring will modulate this laser
        self._brightness[which_laser] = which_ring

    @property
    def laser_lines(self):
        return self._laser_lines

    @property
    def brightness(self):
        return self._brightness

    def __str__(self) -> str:
        return f"Lines: {self.laser_lines}, Brightness: {self.brightness}"
    
    def __repr__(self) -> str:
        return self.__str__()


class RingFSM(object):

    states = ['idle', 'cold', 'search', 'lock']

    def __init__(self,
                 resonances,
                 lasers,
                 initial_loc=0,
                 previous_ring=None,  # None if there's no previous ring, otherwise supply the ring idx
                 cool_down_by=INITIAL_COOLDOWN,  # cool down by this amt in GHz
                 step_size=STEP_SIZE,
                 ) -> None:
        self.laser_locs_seen = []
        self.laser_brightness_seen = []
        self.laser_indices_seen = []
        self.loc = initial_loc  # location of the resonance

        self.locked_laser_idx = None

        self._initial_loc = initial_loc
        self._previous_ring = previous_ring
        self._initial_resonances = resonances
        self._lasers = lasers

        self._cool_down_by = cool_down_by
        self._step_size = step_size
        self._found_all_lines = False

        self.machine = Machine(
            model=self, states=RingFSM.states, initial='idle')

        # 'start' starts the search algorithm
        self.machine.add_transition(trigger='start',
                                    source='idle',
                                    dest='cold',
                                    after='cool_step')

        # once the ring has cooled, start searching
        self.machine.add_transition(trigger='keep_searching',
                                    source='cold',
                                    dest='search',
                                    after='heat_step')

        # 'reset' puts the ring back to idle from any state
        self.machine.add_transition(trigger='reset',
                                    source='*',
                                    dest='idle',
                                    after='reset_resonances')

        # will 'keep_searching' until we find all the laser lines
        self.machine.add_transition(
            trigger='keep_searching',
            source='search',
            dest='search',
            after='heat_step')

        # lock ring once the laser lines have been found
        self.machine.add_transition(
            trigger='lock_to_line',
            source='search',
            dest='lock',
            after='lock_to_laser',
            conditions=['found_all_lines'])

    def reset_resonances(self):
        # reset everything
        self.loc = self._initial_loc
        self.laser_locs_seen = []
        self.laser_brightness_seen = []
        self.laser_indices_seen = []
        self._found_all_lines = False

    def cool(self, amount):
        self.loc += amount

    def heat(self, amount):
        self.loc -= amount

    def cool_step(self):
        # cool the resonances by the cool down by amount
        self.cool(self._cool_down_by)

    def heat_step(self):
        # check if there's a laser signal to be found
        found = self._check_laser_signal()
        # found is tuple (bool, idx | None)
        # if found: (True, laser idx), and if not found: (False, None)
        if found[0]:
            self.laser_locs_seen.append(self.loc)
            self.laser_brightness_seen.append(
                self._lasers.brightness[found[1]])
            self.laser_indices_seen.append(found[1])

        # check if we have found all lasers
        self._check_exit_search_condition()

        # move the resonance location by specified step_size
        self.heat(self._step_size)

    def lock_to_laser(self):
        # jump to the laser signal we want to capture
        self.loc = self.laser_locs_seen[-1]
        # check and capture that laser signal
        self._check_laser_signal(capture=True)

    def _check_laser_signal(self, capture=False):
        # check if there's any laser signal at this location
        found_laser = False
        idx = None
        for r in self.resonances:
            if (r in self._lasers.laser_lines):
                # get the index
                idx = np.where(self._lasers.laser_lines == r)
                assert len(idx) == 1
                idx = idx[0][0]
                if capture:
                    if self._previous_ring is None:
                        this_ring = 0
                    else:
                        this_ring = self._previous_ring + 1
                    self._lasers.capture_laser_line(idx, this_ring)
                    self.locked_laser_idx = idx
                found_laser = True
        # if there's a laser there, found_laser will return that laser's index
        return (found_laser, idx)

    def _check_exit_search_condition(self):
        # if we have seen one dim and one bright, we have found all lines
        if self._previous_ring is None:  # this is the first ring
            if len(self.laser_locs_seen) >= 1:
                self._found_all_lines = True
        else:
            if len(self.laser_locs_seen) >= 2:
                if self.laser_brightness_seen[-2] == self._previous_ring \
                        and self.laser_brightness_seen[-1] == 'bright':
                    self._found_all_lines = True

    @property
    def found_all_lines(self):
        return self._found_all_lines

    @property
    def resonances(self):
        return self.loc + self._initial_resonances

In [28]:
from collections import deque
import copy


def populate_mrm_resonances(global_variations=False,
                            variations=False,
                            corners=False):

    mrm_lines = np.arange(NUM_OF_LASERS, dtype=np.int64) * MRM_FREQ_SPACING
    mrm_lines = np.vstack([mrm_lines] * len(WHICH_RESONANCES)).T

    fsr_shifts = np.array(WHICH_RESONANCES) * MRM_FSR
    mrm_lines += fsr_shifts

    if variations:  # assume we keep the FSR to be the same
        if corners:  # only evaluate the corners
            delta = np.random.randint(2, size=NUM_OF_LASERS)
            delta = 2*delta - 1  # convert to [-1, +1] random numbers
            delta *= MRM_FREQ_UNCERTAINTY
        else:
            delta = np.random.uniform(-1*MRM_FREQ_UNCERTAINTY,
                                      MRM_FREQ_UNCERTAINTY,
                                      NUM_OF_LASERS)
        # round to the nearest STEP_SIZE
        delta = np.round(delta/STEP_SIZE, decimals=0) * STEP_SIZE

        delta = delta.astype(int)
        delta = delta[:, np.newaxis]
        mrm_lines += delta

    if global_variations:  # the global variation can be as much as ±1 FSR
        global_delta = np.random.uniform(-1*MRM_FSR, MRM_FSR)
        global_delta = np.round(global_delta/STEP_SIZE, decimals=0) * STEP_SIZE
        global_delta = global_delta.astype(int)
        mrm_lines += global_delta

    return mrm_lines


def check_barrel_roll(list1, list2):
    # check for barrel roll in sequence or reverse sequence
    assert len(list1) == len(list2)

    def check_equivalent(list1, list2):
        d1 = deque(list1)
        d2 = deque(list2)

        equivalent = d1 == d2

        for _ in range(len(list1)):
            d1.rotate(1)
            if d1 == d2:
                equivalent = True

        return equivalent
    
    list1_ = copy.deepcopy(list1)
    equivalent = check_equivalent(list1_, list2)
    list1_.reverse()
    equivalent = equivalent or check_equivalent(list1_, list2)
    return equivalent

In [29]:
# Try out the algorithm with a single try
lasers = Lasers(variations=True, corners=True, reverse=True, offset=-8*LASER_FREQ_SPACING)
all_ring_resonances = populate_mrm_resonances(variations=True, corners=True)

all_rings = []

for idx in range(NUM_OF_LASERS):
    if idx == 0:
        mrm = RingFSM(resonances=all_ring_resonances[idx], lasers=lasers)
    else:
        mrm = RingFSM(resonances=all_ring_resonances[idx], lasers=lasers, previous_ring=idx-1)
    mrm.start()

    while not mrm.found_all_lines:
        mrm.keep_searching()

    mrm.lock_to_line()
    all_rings.append(mrm)

    print(f"Locked to laser # {mrm.locked_laser_idx}")
    print(lasers.brightness)

lasers_locked_to = [int(m.locked_laser_idx) for m in all_rings]

# check for barrel roll in sequence or reverse sequence
print("Barrel roll check?", check_barrel_roll(lasers_locked_to, BARREL_ROLL_SEQ))

amt_heat_applied = [m.loc for m in all_rings]
print(f"Amount of heat applied: {amt_heat_applied}")

Locked to laser # 4
['bright', 'bright', 'bright', 'bright', 0, 'bright', 'bright', 'bright']
Locked to laser # 0
[1, 'bright', 'bright', 'bright', 0, 'bright', 'bright', 'bright']
Locked to laser # 5
[1, 'bright', 'bright', 'bright', 0, 2, 'bright', 'bright']
Locked to laser # 1
[1, 3, 'bright', 'bright', 0, 2, 'bright', 'bright']
Locked to laser # 6
[1, 3, 'bright', 'bright', 0, 2, 4, 'bright']
Locked to laser # 2
[1, 3, 5, 'bright', 0, 2, 4, 'bright']
Locked to laser # 7
[1, 3, 5, 'bright', 0, 2, 4, 6]
Locked to laser # 3
[1, 3, 5, 7, 0, 2, 4, 6]
Barrel roll check? True
Amount of heat applied: [-50, -850, -1350, -250, -650, -1350, -350, -950]


In [30]:
# Run Monte Carlo and get distribution

import pandas as pd
import copy

eval_corners = False

# cycling_freqs = [-k * MRM_FSR // 9 for k in range(9)]  # change in frequency due to cycling
cycling_freqs = [0]

num_trials = 100
trial_results = []

for _ in range(num_trials):
    lasers = Lasers(variations=True, corners=eval_corners, reverse=True, offset=8*LASER_FREQ_SPACING)
    all_ring_resonances = populate_mrm_resonances(global_variations=True,
                                                  variations=True,
                                                  corners=eval_corners)

    for ctry, cfreq in enumerate(cycling_freqs):

        lasers_ = copy.deepcopy(lasers)
        all_rings = []

        for idx in range(NUM_OF_LASERS):
            if idx == 0:  # the first ring
                mrm = RingFSM(resonances=all_ring_resonances[idx],
                              lasers=lasers_,
                              initial_loc=cfreq)
            else:
                mrm = RingFSM(resonances=all_ring_resonances[idx],
                              lasers=lasers_,
                              initial_loc=cfreq,
                              previous_ring=idx-1)
            # Run the search algorithm for each MRM
            mrm.start()

            while not mrm.found_all_lines:
                mrm.keep_searching()

            mrm.lock_to_line()
            all_rings.append(mrm)

        laser_sequence = [int(m.locked_laser_idx) for m in all_rings]
        amt_heat_applied = [m.loc for m in all_rings]
        colder = [n for n in amt_heat_applied if n >= 0]
        hotter = [n for n in amt_heat_applied if n <= 0]

        max_cool = max(colder) if len(colder) > 0 else 0
        max_heat = min(hotter) if len(hotter) > 0 else 0
        
        max_diff = max_cool - max_heat

        # if True, cycle; else, success and no need for cycling
        cycling_condition = abs(max_heat) > MRM_FSR // 2
        if not cycling_condition:
            break

    result = {'laser_sequence': laser_sequence,
              'barrel_roll': check_barrel_roll(laser_sequence, BARREL_ROLL_SEQ), # check if barrel roll equivalent
              'max_cool': max_cool,
              'max_heat': max_heat,
              'max_diff': max_diff,
              'cycling_trial': ctry,
              'successful_lock': not cycling_condition  # check if we were able to lock with < 1/2 * FSR of heat
              }
    
    ring_numbers = list(range(NUM_OF_LASERS))
    heat_result = dict(zip(ring_numbers, amt_heat_applied))  # (-) value is heat and (+) value is cool
    result.update(heat_result)

    trial_results.append(result)

trial_results = pd.DataFrame(trial_results)



In [31]:
# Now plot the results of the Monte Carlo runs
import plotly.express as px

# Heat and cool amount
fig = px.histogram(np.array(trial_results[range(NUM_OF_LASERS)]).reshape(-1))  # turn into a vector to wash out the ring number
fig.update_traces(xbins_size = 75)
fig.update_layout(xaxis_title="Heat (-) & Cool (+) amount (GHz)", yaxis_title="Count", showlegend=False)
tick_size = 100
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(-MRM_FSR, MRM_FSR+tick_size, tick_size))
    )
)
fig.show()

# Plot the maximum differences
fig = px.histogram(trial_results, x="max_diff")  # turn into a vector to wash out the ring number
fig.update_traces(xbins_size = 75)
fig.update_layout(xaxis_title="Heat amount difference between rings (GHz)", yaxis_title="Count", showlegend=False)
tick_size = 100
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(0, 2*MRM_FSR+tick_size, tick_size))
    )
)
fig.show()

In [25]:
# check if all the results in barrel-roll sequence

print("All in barrel-roll sequence? ", trial_results['barrel_roll'].all())


All in barrel-roll sequence?  True


In [26]:
# display all the trial results in a markdown table
from IPython.display import display, Markdown
display(Markdown(trial_results.to_markdown()))

|    | laser_sequence           | barrel_roll   |   max_cool |   max_heat |   max_diff |   cycling_trial | successful_lock   |    0 |    1 |     2 |     3 |    4 |     5 |     6 |    7 |
|---:|:-------------------------|:--------------|-----------:|-----------:|-----------:|----------------:|:------------------|-----:|-----:|------:|------:|-----:|------:|------:|-----:|
|  0 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         10 |      -1720 |       1730 |               0 | False             |   10 | -570 | -1060 | -1720 | -510 | -1030 | -1710 | -470 |
|  1 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        150 |      -1730 |       1880 |               0 | False             |  150 | -380 | -1110 | -1650 | -440 | -1070 | -1730 | -520 |
|  2 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        160 |      -1670 |       1830 |               0 | False             |  160 | -620 | -1040 | -1640 | -460 | -1200 | -1670 | -580 |
|  3 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |      -1710 |       1710 |               0 | False             |  -30 | -430 | -1150 | -1710 | -630 | -1040 |  -160 | -770 |
|  4 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        200 |      -1710 |       1910 |               0 | False             |  200 | -450 | -1100 | -1710 | -630 | -1250 |  -130 | -750 |
|  5 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        150 |      -1980 |       2130 |               0 | False             |  150 | -520 | -1140 | -1770 | -470 | -1040 | -1980 | -780 |
|  6 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         20 |      -1720 |       1740 |               0 | False             |   20 | -540 | -1190 | -1720 | -460 | -1210 |     0 | -830 |
|  7 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        170 |      -1370 |       1540 |               0 | False             |  170 | -730 | -1330 |    20 | -710 | -1370 |    10 | -760 |
|  8 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        110 |      -1500 |       1610 |               0 | False             |  110 | -610 | -1210 |    50 | -840 | -1500 |  -170 | -890 |
|  9 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        100 |      -1380 |       1480 |               0 | False             |  100 | -590 | -1250 |   -60 | -760 | -1380 |  -100 | -760 |
| 10 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         30 |      -1330 |       1360 |               0 | False             |   30 | -740 | -1330 |   -80 | -770 | -1240 |  -190 | -670 |
| 11 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        170 |      -1410 |       1580 |               0 | False             |  170 | -470 | -1350 |   -60 | -630 | -1410 |  -180 | -740 |
| 12 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         80 |      -1740 |       1820 |               0 | False             |   80 | -460 | -1100 | -1740 | -510 | -1070 | -1610 | -500 |
| 13 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        150 |      -1510 |       1660 |               0 | False             |  150 | -300 |  -860 | -1510 | -250 | -1130 |    20 | -510 |
| 14 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         40 |      -1400 |       1440 |               0 | False             |   40 | -760 | -1330 |   -30 | -720 | -1400 |  -120 | -690 |
| 15 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         90 |      -1660 |       1750 |               0 | False             |   90 | -310 | -1050 | -1660 | -600 | -1220 |   -70 | -590 |
| 16 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        130 |      -1670 |       1800 |               0 | False             |  130 | -450 | -1150 |    40 | -560 | -1200 | -1670 | -580 |
| 17 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        140 |      -1750 |       1890 |               0 | False             |  140 | -570 | -1120 | -1750 | -590 | -1260 |  -130 | -620 |
| 18 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        150 |      -1740 |       1890 |               0 | False             |  150 | -590 | -1170 | -1740 | -440 | -1090 |  -280 | -680 |
| 19 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         80 |      -1880 |       1960 |               0 | False             |   80 | -480 | -1040 | -1880 | -690 | -1290 |   -30 | -640 |
| 20 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |      -1780 |       1780 |               0 | False             |  -70 | -720 | -1320 | -1780 | -740 | -1270 |   -60 | -560 |
| 21 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        140 |      -1350 |       1490 |               0 | False             |  140 | -490 | -1350 |  -140 | -730 | -1280 |  -160 | -620 |
| 22 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |      -1420 |       1420 |               0 | False             | -140 | -770 | -1420 |  -190 | -700 | -1290 |  -150 | -870 |
| 23 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        180 |      -1760 |       1940 |               0 | False             |  180 | -560 | -1160 | -1760 | -580 |  -930 | -1620 | -500 |
| 24 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        110 |      -1290 |       1400 |               0 | False             |  110 | -680 | -1290 |   -70 | -650 | -1180 |    10 | -570 |
| 25 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        160 |      -1850 |       2010 |               0 | False             |  160 | -520 | -1050 | -1660 | -530 | -1130 | -1850 | -670 |
| 26 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        140 |      -1360 |       1500 |               0 | False             |  140 | -380 | -1360 |   -10 | -620 | -1200 |  -140 | -670 |
| 27 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |          0 |      -1390 |       1390 |               0 | False             |  -30 | -560 | -1210 |   -90 | -600 | -1390 |  -260 | -770 |
| 28 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         40 |      -1750 |       1790 |               0 | False             |   40 | -610 | -1110 | -1750 | -670 | -1150 |  -110 | -660 |
| 29 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        110 |      -1770 |       1880 |               0 | False             |  110 | -290 | -1020 | -1510 | -300 | -1200 | -1770 | -580 |
| 30 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         80 |      -1710 |       1790 |               0 | False             |   80 | -500 | -1070 | -1690 | -330 | -1030 | -1710 | -600 |
| 31 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |      -1810 |       1810 |               0 | False             |  -30 | -640 | -1270 | -1810 | -710 | -1330 | -1750 | -520 |
| 32 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        120 |      -1370 |       1490 |               0 | False             |  120 | -680 | -1200 |  -130 | -680 | -1370 |   -30 | -620 |
| 33 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        100 |      -1750 |       1850 |               0 | False             |  100 | -620 | -1210 |   -70 | -680 | -1200 | -1750 | -820 |
| 34 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |      -1420 |       1420 |               0 | False             |  -90 | -860 | -1420 |  -180 | -840 | -1310 |  -210 | -770 |
| 35 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        160 |      -1850 |       2010 |               0 | False             |  160 | -420 | -1020 | -1850 | -690 | -1300 |   -70 | -700 |
| 36 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         30 |      -1730 |       1760 |               0 | False             |   30 | -550 | -1190 | -1730 | -730 | -1420 |  -120 | -690 |
| 37 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        150 |      -1540 |       1690 |               0 | False             |  150 | -310 |  -970 | -1460 | -350 |  -990 | -1540 | -320 |
| 38 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |          0 |      -1790 |       1790 |               0 | False             |  -20 | -620 | -1160 | -1790 | -520 | -1150 |   -10 | -900 |
| 39 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        120 |      -1710 |       1830 |               0 | False             |  120 | -540 | -1220 | -1710 | -490 | -1490 |  -240 | -690 |
| 40 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        130 |      -1730 |       1860 |               0 | False             |  130 | -600 | -1220 | -1730 | -520 | -1110 | -1660 | -750 |
| 41 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        180 |      -1790 |       1970 |               0 | False             |  180 | -280 | -1110 | -1790 | -500 | -1210 | -1690 | -570 |
| 42 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        200 |      -1830 |       2030 |               0 | False             |  200 | -260 | -1070 | -1780 | -510 | -1090 | -1830 | -490 |
| 43 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         70 |      -1850 |       1920 |               0 | False             |   70 | -530 | -1060 | -1850 | -640 | -1410 |  -220 | -740 |
| 44 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         60 |      -1490 |       1550 |               0 | False             |   60 | -790 | -1380 |  -190 | -800 | -1490 |  -200 | -740 |
| 45 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        190 |      -1400 |       1590 |               0 | False             |  190 | -490 | -1400 |  -190 | -730 | -1390 |   -80 | -680 |
| 46 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         80 |      -1270 |       1350 |               0 | False             |   80 | -590 | -1150 |   -50 | -600 | -1270 |  -100 | -880 |
| 47 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |         60 |      -1720 |       1780 |               0 | False             |   60 | -630 | -1200 | -1720 | -800 | -1260 |   -10 | -780 |
| 48 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        170 |      -1760 |       1930 |               0 | False             |  170 | -360 | -1030 | -1590 | -400 |  -930 | -1760 | -600 |
| 49 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        190 |      -1730 |       1920 |               0 | False             |  190 | -500 |  -990 | -1730 | -660 | -1260 |   -80 | -730 |
| 50 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        120 |      -1460 |       1580 |               0 | False             |  120 | -460 | -1080 |   -90 | -700 | -1460 |  -190 | -750 |
| 51 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |      -1580 |       1580 |               0 | False             | -210 | -960 | -1410 |  -200 | -870 | -1580 |  -130 | -900 |
| 52 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |         90 |      -1330 |       1420 |               0 | False             |   90 | -590 | -1180 |   -50 | -510 | -1330 |  -170 | -790 |
| 53 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         90 |      -1680 |       1770 |               0 | False             |   90 | -390 |  -960 | -1680 | -340 |  -990 | -1630 | -630 |
| 54 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         60 |      -1350 |       1410 |               0 | False             |   60 | -770 | -1350 |   -70 | -760 | -1290 |   -90 | -730 |
| 55 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         80 |      -1380 |       1460 |               0 | False             |   80 | -740 | -1300 |  -220 | -740 | -1380 |  -230 | -790 |
| 56 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        180 |      -1360 |       1540 |               0 | False             |  180 | -450 | -1360 |   -90 | -720 | -1170 |  -140 | -630 |
| 57 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        200 |      -1710 |       1910 |               0 | False             |  200 | -540 | -1050 | -1670 | -440 | -1090 | -1710 | -580 |
| 58 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        150 |      -1970 |       2120 |               0 | False             |  150 | -570 | -1160 | -1640 | -510 | -1070 | -1970 | -700 |
| 59 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        150 |      -1700 |       1850 |               0 | False             |  150 | -430 | -1130 | -1650 | -610 | -1160 | -1700 | -570 |
| 60 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |          0 |      -1690 |       1690 |               0 | False             |  -50 | -400 | -1140 | -1690 | -490 | -1070 |  -240 | -770 |
| 61 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         40 |      -1870 |       1910 |               0 | False             |   40 | -560 | -1230 |     0 | -610 | -1060 | -1870 | -660 |
| 62 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         30 |      -1920 |       1950 |               0 | False             |   30 | -410 | -1090 | -1580 | -550 |  -990 | -1920 | -630 |
| 63 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        120 |      -1820 |       1940 |               0 | False             |  120 | -400 |  -980 | -1820 | -700 | -1120 |   -30 | -520 |
| 64 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |      -1320 |       1320 |               0 | False             |  -40 | -650 | -1320 |  -110 | -730 | -1180 |   -20 | -590 |
| 65 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        120 |      -1260 |       1380 |               0 | False             |  120 | -390 | -1260 |   -10 | -680 | -1240 |   -20 | -590 |
| 66 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        170 |      -1580 |       1750 |               0 | False             |  170 | -400 |  -990 | -1580 | -350 |  -950 | -1570 | -470 |
| 67 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        120 |      -1270 |       1390 |               0 | False             |  120 | -670 | -1270 |   -20 | -500 | -1190 |  -200 | -700 |
| 68 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         30 |      -1170 |       1200 |               0 | False             |   30 | -670 | -1150 |  -140 | -740 | -1170 |   -80 | -570 |
| 69 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        140 |      -1830 |       1970 |               0 | False             |  140 | -580 | -1110 | -1830 | -550 | -1250 |   -90 | -540 |
| 70 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        110 |      -1700 |       1810 |               0 | False             |  110 | -530 | -1020 | -1700 | -570 | -1100 | -1670 | -480 |
| 71 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         40 |      -1370 |       1410 |               0 | False             |   40 | -710 | -1310 |   -10 | -700 | -1370 |  -250 | -900 |
| 72 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |        190 |      -1380 |       1570 |               0 | False             |  190 | -800 | -1330 |  -140 | -780 | -1380 |  -160 | -760 |
| 73 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        170 |      -1770 |       1940 |               0 | False             |  170 | -390 | -1150 |   -40 | -570 | -1210 | -1770 | -520 |
| 74 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        150 |      -1830 |       1980 |               0 | False             |  150 | -470 | -1090 | -1450 | -270 |  -920 | -1830 | -640 |
| 75 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         70 |      -1810 |       1880 |               0 | False             |   70 | -510 | -1120 | -1730 | -640 | -1200 | -1810 | -420 |
| 76 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        140 |      -1840 |       1980 |               0 | False             |  140 | -380 | -1100 |  -120 | -740 | -1090 | -1840 | -710 |
| 77 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         60 |      -1300 |       1360 |               0 | False             |   60 | -570 | -1300 |   -50 | -670 | -1290 |   -20 | -750 |
| 78 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |         50 |      -1560 |       1610 |               0 | False             |   50 | -720 | -1420 |  -120 | -810 | -1560 |  -140 | -720 |
| 79 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |        100 |      -1670 |       1770 |               0 | False             |  100 | -510 | -1180 | -1670 | -450 | -1250 |   -30 | -470 |
| 80 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |      -1830 |       1830 |               0 | False             |  -20 | -580 | -1340 |   -10 | -720 | -1270 | -1830 | -710 |
| 81 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        110 |      -1760 |       1870 |               0 | False             |  110 | -630 | -1250 | -1760 | -670 | -1270 |  -200 | -930 |
| 82 | [7, 3, 4, 0, 5, 1, 6, 2] | True          |        130 |      -1430 |       1560 |               0 | False             |  130 | -540 | -1390 |  -260 | -710 | -1430 |  -150 | -800 |
| 83 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |         30 |      -1720 |       1750 |               0 | False             |   30 | -370 | -1120 | -1690 | -420 | -1140 | -1720 | -720 |
| 84 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        150 |      -1910 |       2060 |               0 | False             |  150 | -600 | -1170 | -1700 | -560 | -1110 | -1910 | -720 |
| 85 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |         80 |      -1920 |       2000 |               0 | False             |   80 | -630 | -1200 |  -160 | -750 | -1220 | -1920 | -860 |
| 86 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        200 |      -1870 |       2070 |               0 | False             |  200 | -390 | -1020 | -1540 | -470 | -1050 | -1870 | -560 |
| 87 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |          0 |      -1890 |       1890 |               0 | False             |  -40 | -460 | -1210 | -1890 | -690 | -1310 |  -120 | -690 |
| 88 | [3, 4, 0, 5, 1, 6, 2, 7] | True          |         30 |      -1310 |       1340 |               0 | False             |   30 | -750 | -1310 |  -140 | -680 | -1230 |    20 | -590 |
| 89 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |      -1470 |       1470 |               0 | False             |  -80 | -810 | -1470 |   -80 | -890 | -1470 |  -120 | -900 |
| 90 | [0, 5, 1, 6, 2, 7, 3, 4] | True          |        160 |      -1570 |       1730 |               0 | False             |  160 | -350 | -1110 | -1550 | -540 | -1060 | -1570 | -610 |
| 91 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |         50 |      -1990 |       2040 |               0 | False             |   50 | -380 | -1110 | -1990 | -700 | -1250 |   -90 | -590 |
| 92 | [6, 2, 7, 3, 4, 0, 5, 1] | True          |        140 |      -1630 |       1770 |               0 | False             |  140 | -350 |  -980 | -1630 | -650 | -1170 |   -70 | -530 |
| 93 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |          0 |      -1370 |       1370 |               0 | False             |  -50 | -620 | -1370 |  -120 | -610 | -1310 |   -80 | -680 |
| 94 | [4, 0, 5, 1, 6, 2, 7, 3] | True          |         40 |      -1810 |       1850 |               0 | False             |   40 | -610 | -1320 |   -60 | -520 | -1180 | -1810 | -650 |
| 95 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        180 |      -1780 |       1960 |               0 | False             |  180 | -310 |  -880 | -1430 | -290 |  -960 | -1780 | -460 |
| 96 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        120 |      -1850 |       1970 |               0 | False             |  120 | -620 | -1160 | -1850 | -670 | -1390 |   -60 | -720 |
| 97 | [1, 6, 2, 7, 3, 4, 0, 5] | True          |        180 |      -1620 |       1800 |               0 | False             |  180 | -490 | -1070 | -1620 | -610 | -1300 |  -170 | -690 |
| 98 | [5, 1, 6, 2, 7, 3, 4, 0] | True          |        180 |      -1810 |       1990 |               0 | False             |  180 | -460 | -1170 | -1800 | -470 | -1120 | -1810 | -610 |
| 99 | [2, 7, 3, 4, 0, 5, 1, 6] | True          |        140 |      -1820 |       1960 |               0 | False             |  140 | -480 |  -960 | -1820 | -600 | -1100 | -1820 | -700 |